In [171]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Point
from shapely.ops import linemerge
from tqdm import tqdm
from pyproj import Geod

In [16]:
speed = pd.read_csv('data/Daejeon/speed_imputed.csv', index_col='Unnamed: 0')
target = list(speed.columns)

In [120]:
df = gpd.read_file('data/Daejeon/network/Linkshape20220614.shp')
df = df[['TLINKIDP1', 'LENGTH', 'geometry']]
df['LENGTH'] = df['LENGTH'].astype(int)
df = df[df['TLINKIDP1'].notnull()]
df['TLINKIDP1'] = df['TLINKIDP1'].astype(str)
df = df[df['TLINKIDP1'].isin(target)]
df = df.sort_values(by='TLINKIDP1')
df = df.reset_index(drop=True)

In [50]:
unique_values_count = df['TLINKIDP1'].nunique()
print("중복 제거 후 유일한 값의 개수:", unique_values_count)

중복 제거 후 유일한 값의 개수: 387


In [121]:
nex = 1
i = 0
while i <= 1418:
    cnt = 0
    geolist = []
    while df['TLINKIDP1'][i] == df['TLINKIDP1'][nex]:
        df.loc[i,'LENGTH'] += df.loc[nex,'LENGTH']
        geolist.append(df['geometry'][nex])
        nex += 1
        cnt += 1
        if nex == 1419:
            break
    if cnt > 0:
        geolist.insert(0,df['geometry'][i])
        #df.loc[i,'geometry'] = linemerge(geolist)
        df['geometry'][i] = linemerge(geolist)
        i += cnt
        nex += cnt
    else:
        i += 1
        nex = i+1
    if nex >= 1419:
        break

In [122]:
df.drop_duplicates(subset=['TLINKIDP1'], keep='first', inplace=True)
df = df.reset_index(drop=True)

In [131]:
speed = pd.read_csv('data/Daejeon/speed_imputed.csv', index_col='Unnamed: 0')
speed = speed[speed.columns.intersection(df['TLINKIDP1'])]
speed.index = pd.to_datetime(speed.index)
speed.to_hdf('data/Daejeon/Daejeon.h5', 'df', format='table')
speed.to_csv('data/Daejeon/speed_final.csv')

In [124]:
df.to_csv('data/Daejeon/edges.csv')

In [127]:
f = open("data/Daejeon/graph_sensor_Daejeon.txt", "w")
for link in df['TLINKIDP1']:
    f.write(link + ',')
f.close()

In [133]:
df

,TLINKIDP1,LENGTH,geometry
0,10123421,48,"LINESTRING (127.28822 36.46979, 127.28833 36.4..."
1,10123422,613,"LINESTRING (127.29484 36.47205, 127.29390 36.4..."
2,10123431,1431,"MULTILINESTRING ((127.27488 36.46359, 127.2754..."
6,10123432,24,"LINESTRING (127.28014 36.46559, 127.27989 36.4..."
7,10194321,28,"LINESTRING (127.28225 36.25000, 127.28227 36.2..."
...,...,...,...
1406,3444162,318,"MULTILINESTRING ((127.31684 36.33103, 127.3171..."
1409,3444201,244,"LINESTRING (127.47939 36.23543, 127.47976 36.2..."
1410,3444202,1310,"LINESTRING (127.48083 36.23356, 127.47950 36.2..."
1413,3444331,1366,"LINESTRING (127.31055 36.39709, 127.31188 36.3..."


In [186]:
col = ['from', 'to', 'distance']
dist = pd.DataFrame(columns=col)
dist

,from,to,distance


In [188]:
geod = Geod(ellps="WGS84")
for i in tqdm(range(len(df))):
    for j in range(1,len(df)):
        if i != j:
            line = LineString([df['geometry'][i].centroid, df['geometry'][j].centroid])
            di = geod.geometry_length(line)
            dist = dist.append([{'from': df['TLINKIDP1'][i],
                              'to': df['TLINKIDP1'][j],
                              'distance': di}], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 387/387 [02:28<00:00,  2.61it/s]


In [190]:
dist.to_csv('data/Daejeon/distances_daejeon.csv')

In [182]:
df

,TLINKIDP1,LENGTH,geometry
0,10123421,48,"LINESTRING (127.28822 36.46979, 127.28833 36.4..."
1,10123422,613,"LINESTRING (127.29484 36.47205, 127.29390 36.4..."
2,10123431,1431,"MULTILINESTRING ((127.27488 36.46359, 127.2754..."
3,10123432,24,"LINESTRING (127.28014 36.46559, 127.27989 36.4..."
4,10194321,28,"LINESTRING (127.28225 36.25000, 127.28227 36.2..."
...,...,...,...
382,3444162,318,"MULTILINESTRING ((127.31684 36.33103, 127.3171..."
383,3444201,244,"LINESTRING (127.47939 36.23543, 127.47976 36.2..."
384,3444202,1310,"LINESTRING (127.48083 36.23356, 127.47950 36.2..."
385,3444331,1366,"LINESTRING (127.31055 36.39709, 127.31188 36.3..."


In [192]:
speed

,3391442,3402981,3404081,3404102,3434152,3439012,3440371,3442932,3444332,10194442,...,3402982,10202002,3358341,3388262,3402842,16004891,10194532,10424801,3428961,3386892
2022-01-01 00:00:00,58.0,53.0,55.0,79.0,65.0,81.0,48.0,86.0,93.0,106.0,...,52.0,69.0,38.0,23.0,54.0,64.0,89.0,58.0,62.0,12.0
2022-01-01 00:05:00,59.0,52.0,38.0,73.0,73.0,73.0,53.0,93.0,107.0,96.0,...,25.0,69.0,36.0,33.0,52.0,54.0,96.0,55.0,47.0,19.0
2022-01-01 00:10:00,60.0,57.0,51.0,75.0,71.0,58.0,55.0,92.0,113.0,75.0,...,37.0,73.0,33.0,38.0,40.0,54.0,93.0,55.0,42.0,53.0
2022-01-01 00:15:00,60.0,57.0,57.0,76.0,68.0,82.0,54.0,94.0,100.0,77.0,...,26.0,73.0,41.0,24.0,51.0,62.0,94.0,48.0,59.0,24.0
2022-01-01 00:20:00,57.0,54.0,55.0,74.0,68.0,82.0,52.0,96.0,108.0,77.0,...,15.0,63.0,15.0,25.0,16.0,50.0,92.0,52.0,68.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 23:35:00,58.0,55.0,39.0,74.0,61.0,76.0,47.0,92.0,98.0,77.0,...,31.0,62.0,34.0,29.0,53.0,63.0,89.0,47.0,55.0,17.0
2022-06-30 23:40:00,57.0,7.0,44.0,76.0,58.0,87.0,49.0,89.0,100.0,93.0,...,27.0,60.0,43.0,37.0,10.0,60.0,95.0,53.0,34.0,27.0
2022-06-30 23:45:00,59.0,24.0,41.0,74.0,60.0,89.0,50.0,86.0,105.0,93.0,...,26.0,64.0,39.0,28.0,35.0,60.0,94.0,51.0,54.0,27.0
2022-06-30 23:50:00,59.0,39.0,38.0,77.0,56.0,90.0,50.0,93.0,96.0,93.0,...,23.0,70.0,15.0,41.0,45.0,60.0,89.0,53.0,56.0,19.0


In [199]:
len(speed.columns)

387

In [198]:
speed.to_hdf('data/Daejeon/Daejeon.h5', 'df', format='table')